In [1]:
import pandas as pd
from openai import AzureOpenAI
import json
import openai
from prompts import *
import pickle

In [2]:
employment_ground_truth = pd.read_csv('eval_employment_prompt.csv')
relation_ground_truth = pd.read_csv('eval_relationship_prompt.csv')

In [3]:
employment_ground_truth.head()

,index,provider_type,patient_id,note_id,sentence_index,text,TRANSPORTATION_distance,TRANSPORTATION_resource,TRANSPORTATION_other,HOUSING_poor,...,EMPLOYMENT_student,SUPPORT_plus,SUPPORT_minus,PARENT,UNKNOWN,llm_text,llm_label_EMPLOYMENT,llm_label_UNKNOWN,EMPLOYMENT_nonadverse,EMPLOYMENT_adverse
0,0,Nursing,10954,401903,0,Ms. [**Known lastname 11692**] is a 52 year ol...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False
1,1,Nursing,10954,401903,1,"During her admission, she developed obtundatio...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False
2,2,Nursing,10954,401903,2,She was extubated the next day after a thorace...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False
3,3,Nursing,10954,401903,3,She was transferred to the medical service fro...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False
4,4,Nursing,10954,401903,4,"Please see Vascular, SICU and Medicine admissi...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False


In [4]:
relation_ground_truth.head()

,index,provider_type,patient_id,note_id,sentence_index,text,TRANSPORTATION_distance,TRANSPORTATION_resource,TRANSPORTATION_other,HOUSING_poor,...,SUPPORT_plus,SUPPORT_minus,PARENT,UNKNOWN,llm_text,llm_label_marital_status,llm_label_RELATIONSHIP,llm_label_UNKNOWN,RELATIONSHIP_nonadverse,RELATIONSHIP_adverse
0,0,Nursing,10954,401903,0,Ms. [**Known lastname 11692**] is a 52 year ol...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,"{\n ""marital_status"": ""UNKNOWN"",\n ""marital_...",UNKNOWN,UNKNOWN,True,False,False
1,1,Nursing,10954,401903,1,"During her admission, she developed obtundatio...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,"{\n ""marital_status"": ""UNKNOWN"",\n ""marital_...",UNKNOWN,UNKNOWN,True,False,False
2,2,Nursing,10954,401903,2,She was extubated the next day after a thorace...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,"{\n ""marital_status"": ""UNKNOWN"",\n ""marital_...",UNKNOWN,UNKNOWN,True,False,False
3,3,Nursing,10954,401903,3,She was transferred to the medical service fro...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,"{\n ""marital_status"": ""UNKNOWN"",\n ""marital_...",UNKNOWN,UNKNOWN,True,False,False
4,4,Nursing,10954,401903,4,"Please see Vascular, SICU and Medicine admissi...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,"{\n ""marital_status"": ""UNKNOWN"",\n ""marital_...",UNKNOWN,UNKNOWN,True,False,False


In [5]:
employment_ground_truth.columns

Index(['index', 'provider_type', 'patient_id', 'note_id', 'sentence_index',
       'text', 'TRANSPORTATION_distance', 'TRANSPORTATION_resource',
       'TRANSPORTATION_other', 'HOUSING_poor', 'HOUSING_undomiciled',
       'HOUSING_other', 'RELATIONSHIP_married', 'RELATIONSHIP_partnered',
       'RELATIONSHIP_divorced', 'RELATIONSHIP_widowed', 'RELATIONSHIP_single',
       'EMPLOYMENT_employed', 'EMPLOYMENT_underemployed',
       'EMPLOYMENT_unemployed', 'EMPLOYMENT_disability', 'EMPLOYMENT_retired',
       'EMPLOYMENT_student', 'SUPPORT_plus', 'SUPPORT_minus', 'PARENT',
       'UNKNOWN', 'llm_text', 'llm_label_EMPLOYMENT', 'llm_label_UNKNOWN',
       'EMPLOYMENT_nonadverse', 'EMPLOYMENT_adverse'],
      dtype='object')

In [6]:
relation_ground_truth.columns

Index(['index', 'provider_type', 'patient_id', 'note_id', 'sentence_index',
       'text', 'TRANSPORTATION_distance', 'TRANSPORTATION_resource',
       'TRANSPORTATION_other', 'HOUSING_poor', 'HOUSING_undomiciled',
       'HOUSING_other', 'RELATIONSHIP_married', 'RELATIONSHIP_partnered',
       'RELATIONSHIP_divorced', 'RELATIONSHIP_widowed', 'RELATIONSHIP_single',
       'EMPLOYMENT_employed', 'EMPLOYMENT_underemployed',
       'EMPLOYMENT_unemployed', 'EMPLOYMENT_disability', 'EMPLOYMENT_retired',
       'EMPLOYMENT_student', 'SUPPORT_plus', 'SUPPORT_minus', 'PARENT',
       'UNKNOWN', 'llm_text', 'llm_label_marital_status',
       'llm_label_RELATIONSHIP', 'llm_label_UNKNOWN',
       'RELATIONSHIP_nonadverse', 'RELATIONSHIP_adverse'],
      dtype='object')

In [7]:
relation_df = relation_ground_truth[['RELATIONSHIP_nonadverse', 'RELATIONSHIP_adverse']]

In [8]:
final_df = employment_ground_truth.join(relation_df, on='index', how='left')

In [9]:
final_df.head()

,index,provider_type,patient_id,note_id,sentence_index,text,TRANSPORTATION_distance,TRANSPORTATION_resource,TRANSPORTATION_other,HOUSING_poor,...,SUPPORT_minus,PARENT,UNKNOWN,llm_text,llm_label_EMPLOYMENT,llm_label_UNKNOWN,EMPLOYMENT_nonadverse,EMPLOYMENT_adverse,RELATIONSHIP_nonadverse,RELATIONSHIP_adverse
0,0,Nursing,10954,401903,0,Ms. [**Known lastname 11692**] is a 52 year ol...,0.0,0.0,0.0,0.0,...,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False,False,False
1,1,Nursing,10954,401903,1,"During her admission, she developed obtundatio...",0.0,0.0,0.0,0.0,...,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False,False,False
2,2,Nursing,10954,401903,2,She was extubated the next day after a thorace...,0.0,0.0,0.0,0.0,...,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False,False,False
3,3,Nursing,10954,401903,3,She was transferred to the medical service fro...,0.0,0.0,0.0,0.0,...,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False,False,False
4,4,Nursing,10954,401903,4,"Please see Vascular, SICU and Medicine admissi...",0.0,0.0,0.0,0.0,...,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False,False,False


In [10]:
with open('azure_credentials.json', 'r') as file:
    azure_data = json.load(file)
    api_key = azure_data['API_KEY']
    api_version = azure_data['API_VERSION']
    azure_endpoint = azure_data['AZURE_ENDPOINT']
    azure_deployment_name = azure_data['AZURE_DEPLOYMENT_NAME']

client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint = azure_endpoint
    )

deployment_name=azure_deployment_name #This will correspond to the custom name you chose for your deployment when you deployed a model. Use a gpt-35-turbo-instruct deployment.

# Defining a function to create the prompt from the instruction system message, the few-shot examples, and the current query
def create_prompt(system_message, user_message):    
    formatted_message = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
    
    return formatted_message

# This function sends the prompt to the GPT model
def send_message(message, model_name, max_response_tokens=500):
    response = client.chat.completions.create(
        model=model_name,
        messages=message,
        temperature=0,
        max_tokens=max_response_tokens,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    return response.choices[0].message.content.strip()

In [11]:
final_df['EMPLOYMENT_nonadverse'].value_counts()

EMPLOYMENT_nonadverse
False    5276
True       45
Name: count, dtype: int64

In [11]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['EMPLOYMENT_nonadverse'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_employment_nonadverse_step1 = pd.DataFrame({'index': index_list, 'llm_employment_nonadverse': llm_response_list})

with open('llm_employment_nonadverse_step1.pkl', 'wb') as file:
    pickle.dump(llm_employment_nonadverse_step1, file)

Stopped Seroquel n SICU - n case exacerbates arrhytmias Patient admitted from: [**Hospital1 5**] OR / PACU, SICU History obtained from Patient, Family / Friend, [**Hospital 961**] Medical records Allergies: No Known Drug Allergies Last dose of Antibiotics: Vancomycin - [**2180-8-21**] 08:14 AM Infusions: Other ICU medications: Other medications: Past medical history: Family history: Social History: Stage IV Lung Ca - brain mets s/p right crani at [**Hospital 1215**] in [**9-5**] rx with chrmo XRT without good response and currently off all Rx PE [**1-7**] on coumadin Bipolar- hx of hospitalization x 2 for mania DM GERD Occupation: auto detailer car sales Drugs: neg Tobacco: 40 pk yrs Alcohol: neg Other: lives in [**Location (un) 1130**] with family Review of systems: Cardiovascular: Tachycardia Respiratory: Cough Gastrointestinal: No(t) Abdominal pain Genitourinary: Foley Heme / Lymph: Anemia Neurologic: Seizure Flowsheet Data as of [**2180-8-22**] 02:05 PM Vital Signs Hemodynamic moni

In [14]:
final_df['EMPLOYMENT_adverse'].value_counts()

EMPLOYMENT_adverse
False    5299
True       22
Name: count, dtype: int64

In [12]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['EMPLOYMENT_adverse'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_employment_adverse_step1 = pd.DataFrame({'index': index_list, 'llm_employment_adverse': llm_response_list})

with open('llm_employment_adverse_step1.pkl', 'wb') as file:
    pickle.dump(llm_employment_adverse_step1, file)

He has currently been doing temp work but has had no recent steady work.
YES

Social Work SW spoke with pt s HR rep ([**Telephone/Fax (1) 11673**]) at [**Company 1855**] this AM who repots that pt s short term disability needs to be reinitiated by hospital getting in touch with [**Location (un) 11674**] Insurance CO.
YES

Past psychiatric history: None documented Past addictions history: None known Employment status: Disable Legal involvement: None known Additional Information: Patient / Family History: (Meeting with pt's husband, son, and daughter.) A doctor admitted this 74 y/o MWF for sepsis; she has end stage renal disease.
YES

[**Name (NI) **] Communication or visitation restriction: Patient Information: Previous living situation: Previous level of functioning: Independent Previous [**Hospital1 54**] or other hospital admissions: Pt with h/o epilepsy, no previous admits at [**Hospital1 54**] Past psychiatric history: none known Past addictions history: Pt with h/o ETOH abuse Empl

In [15]:
final_df['RELATIONSHIP_nonadverse'].value_counts()

RELATIONSHIP_nonadverse
False    5157
True      157
Name: count, dtype: int64

In [16]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['RELATIONSHIP_nonadverse'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_relationship_nonadverse_step1 = pd.DataFrame({'index': index_list, 'llm_relationship_nonadverse': llm_response_list})

with open('llm_relationship_nonadverse_step1.pkl', 'wb') as file:
    pickle.dump(llm_relationship_nonadverse_step1, file)

Pts husband in most ot the night.
YES

Morphine drip added.
NO

Endo: blood sugars 233 and 207 Social: Pt s son called in AM updated on pt s condition.
YES

We are to cont with collagenase.
NO

51M with HCV/HCC s/p liver transplant [**12-18**] (discharged [**12-24**]).  2 days prior to admission patient increasing experienced  confusion and eventually was admitted to the transplant surgical service [**12-29**].
NO

66 year old man with PMHx lung cancer 6 yrs ago, s/p RUL lobectomy (no chemo or XRT), as well as spondolithiasis s/p two laminectomies.
YES

Plan: Patient needs unit of PRBC s.  Continue to monitor for bleed.
NO

[**Hospital1 **] for pancytopenia, elevated LFTs and GIB with HCT 23 at OSH, trasnfused 1 unit.
NO

Pt reports he drinks 1 glass of wine and 3 [**Location 8936**] per day which has been unchanged for years.
NO

Some crusting on lips after application of orabase.
NO

- restart Carvedilol once BP stable - holding home Torsemide given ARF and then lasix gtt.
NO

- Hold

In [ ]:
final_df['RELATIONSHIP_adverse'].value_counts()

RELATIONSHIP_adverse
False    5291
True       23
Name: count, dtype: int64

In [ ]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['RELATIONSHIP_adverse'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_relationship_adverse_step1 = pd.DataFrame({'index': index_list, 'llm_relationship_adverse': llm_response_list})

with open('llm_relationship_adverse_step1.pkl', 'wb') as file:
    pickle.dump(llm_relationship_adverse_step1, file)

In [12]:
final_df['TRANSPORTATION_distance'].value_counts()

TRANSPORTATION_distance
0.0    5319
1.0       2
Name: count, dtype: int64

In [13]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['TRANSPORTATION_distance'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_transportation_distance_step1 = pd.DataFrame({'index': index_list, 'llm_transportation_distance': llm_response_list})

with open('llm_transportation_distance_step1.pkl', 'wb') as file:
    pickle.dump(llm_transportation_distance_step1, file)

They are hoping to relocate pt from his current living situation in [**Location (un) 702**], MA closer to their home in NH.
YES

Wife also reports that she prefers "The Maples" [**Hospital1 238**] near [**Location (un) 13062**] if possible because she lives in [**Location 5548**].
YES



In [17]:
final_df['TRANSPORTATION_resource'].value_counts()

TRANSPORTATION_resource
0.0    5321
Name: count, dtype: int64

In [18]:
final_df['TRANSPORTATION_other'].value_counts()

TRANSPORTATION_other
0.0    5320
1.0       1
Name: count, dtype: int64

In [66]:
final_df['HOUSING_poor'].value_counts()

HOUSING_poor
0.0    5319
1.0       2
Name: count, dtype: int64

In [19]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['HOUSING_poor'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_housing_poor_step1 = pd.DataFrame({'index': index_list, 'llm_housing_poor': llm_response_list})

with open('llm_housing_poor_step1.pkl', 'wb') as file:
    pickle.dump(llm_housing_poor_step1, file)

The combined income of both allows them to pay rent in subsidized housing but the couple does live from month to month.
YES

Per previous OMR note, pt lives alone in an apartment at [**Location 7455**] House ([**0-0-**]), a subsidized independent living building for seniors and adults with disabilities in
YES



In [22]:
final_df['HOUSING_undomiciled'].value_counts()

HOUSING_undomiciled
0.0    5321
Name: count, dtype: int64

In [23]:
final_df['HOUSING_other'].value_counts()

HOUSING_other
0.0    5320
1.0       1
Name: count, dtype: int64

In [24]:
final_df['RELATIONSHIP_married'].value_counts()

RELATIONSHIP_married
0.0    5189
1.0     132
Name: count, dtype: int64

In [25]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['RELATIONSHIP_married'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_relationship_married_step1 = pd.DataFrame({'index': index_list, 'llm_relationship_married': llm_response_list})

with open('llm_relationship_married_step1.pkl', 'wb') as file:
    pickle.dump(llm_relationship_married_step1, file)

Pts husband in most ot the night.
YES

Pt husband, daughter, 2 sisters and grandson at her bedside.
YES

Sons and husband in to visit today, spoke with physician [**Name Initial (PRE) 683**]/Response: Pt had very large soft, non-formed BM, light brown, guiac positive.
YES

Plan: Continue q2h turns, skin care, nutrition; daily dressing changes to coccyx with collagenase debridement. [**Name (NI) 1457**] wife called and was updated by this nurse this am.
NO

Russian is his first language but wife states pt. speaks and understands English.
YES

He lives in [**State 5568**], although he and his wife have been in the process of moving back to [**Location (un) 23**].
YES

Social: patient s wife plans to stay in cot in room overnight.
YES

Reportedly, pt last seen by husband at midnight last night, "passed out" in a chair,  presumed was due to EtOH.
YES

Pt reportedly snoring overnight, then when stopped snoring this AM husband called 911 (also found suicide note).
YES

Airway, Inability to P

In [ ]:
final_df['RELATIONSHIP_partnered'].value_counts()

RELATIONSHIP_partnered
0.0    5296
1.0      25
Name: count, dtype: int64

In [ ]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['RELATIONSHIP_partnered'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_relationship_partnered_step1 = pd.DataFrame({'index': index_list, 'llm_relationship_partnered': llm_response_list})

with open('llm_relationship_partnered_step1.pkl', 'wb') as file:
    pickle.dump(llm_relationship_partnered_step1, file)

In [ ]:
final_df['RELATIONSHIP_divorced'].value_counts()

RELATIONSHIP_divorced
0.0    5304
1.0      17
Name: count, dtype: int64

In [ ]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['RELATIONSHIP_divorced'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_relationship_divorced_step1 = pd.DataFrame({'index': index_list, 'llm_relationship_divorced': llm_response_list})

with open('llm_relationship_divorced_step1.pkl', 'wb') as file:
    pickle.dump(llm_relationship_divorced_step1, file)

In [ ]:
final_df['RELATIONSHIP_widowed'].value_counts()

RELATIONSHIP_widowed
0.0    5320
1.0       1
Name: count, dtype: int64

In [ ]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['RELATIONSHIP_widowed'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_relationship_widowed_step1 = pd.DataFrame({'index': index_list, 'llm_relationship_widowed': llm_response_list})

with open('llm_relationship_widowed_step1.pkl', 'wb') as file:
    pickle.dump(llm_relationship_widowed_step1, file)

In [ ]:
final_df['RELATIONSHIP_single'].value_counts()

RELATIONSHIP_single
0.0    5316
1.0       5
Name: count, dtype: int64

In [ ]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['RELATIONSHIP_single'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_relationship_single_step1 = pd.DataFrame({'index': index_list, 'llm_relationship_single': llm_response_list})

with open('llm_relationship_single_step1.pkl', 'wb') as file:
    pickle.dump(llm_relationship_single_step1, file)

In [ ]:
final_df['EMPLOYMENT_employed'].value_counts()

EMPLOYMENT_employed
0.0    5290
1.0      31
Name: count, dtype: int64

In [ ]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['EMPLOYMENT_employed'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_employment_employed_step1 = pd.DataFrame({'index': index_list, 'llm_employment_employed': llm_response_list})

with open('llm_employment_employed_step1.pkl', 'wb') as file:
    pickle.dump(llm_employment_employed_step1, file)

In [ ]:
final_df['EMPLOYMENT_underemployed'].value_counts()

EMPLOYMENT_underemployed
0.0    5318
1.0       3
Name: count, dtype: int64

In [ ]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['EMPLOYMENT_underemployed'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_employment_underemployed_step1 = pd.DataFrame({'index': index_list, 'llm_employment_underemployed': llm_response_list})

with open('llm_employment_underemployed_step1.pkl', 'wb') as file:
    pickle.dump(llm_employment_underemployed_step1, file)

In [ ]:
final_df['EMPLOYMENT_unemployed'].value_counts()

EMPLOYMENT_unemployed
0.0    5311
1.0      10
Name: count, dtype: int64

In [ ]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['EMPLOYMENT_unemployed'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_employment_unemployed_step1 = pd.DataFrame({'index': index_list, 'llm_employment_unemployed': llm_response_list})

with open('llm_employment_unemployed_step1.pkl', 'wb') as file:
    pickle.dump(llm_employment_unemployed_step1, file)

In [ ]:
final_df['EMPLOYMENT_disability'].value_counts()

EMPLOYMENT_disability
0.0    5311
1.0      10
Name: count, dtype: int64

In [ ]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['EMPLOYMENT_disability'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_employment_disability_step1 = pd.DataFrame({'index': index_list, 'llm_employment_disability': llm_response_list})

with open('llm_employment_disability_step1.pkl', 'wb') as file:
    pickle.dump(llm_employment_disability_step1, file)

In [ ]:
final_df['EMPLOYMENT_retired'].value_counts()

EMPLOYMENT_retired
0.0    5316
1.0       5
Name: count, dtype: int64

In [ ]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['EMPLOYMENT_retired'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_employment_retired_step1 = pd.DataFrame({'index': index_list, 'llm_employment_retired': llm_response_list})

with open('llm_employment_retired_step1.pkl', 'wb') as file:
    pickle.dump(llm_employment_retired_step1, file)

In [ ]:
final_df['EMPLOYMENT_student'].value_counts()

EMPLOYMENT_student
0.0    5310
1.0      11
Name: count, dtype: int64

In [ ]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
for index, row in final_df[final_df['EMPLOYMENT_student'] == True].iterrows():
    free_text = row['text']
    user_message = step1_query_optimized.format(free_text=free_text)
    openai_message = create_prompt(system_message, user_message)
    response = send_message(openai_message, deployment_name)
    
    index_list.append(index)
    llm_response_list.append(response)
    print(free_text)
    print(response)
    print()

llm_employment_student_step1 = pd.DataFrame({'index': index_list, 'llm_employment_student': llm_response_list})

with open('llm_employment_student_step1.pkl', 'wb') as file:
    pickle.dump(llm_employment_student_step1, file)

In [12]:
free_text = final_df[final_df['EMPLOYMENT_adverse'] == True].iloc[0].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query_optimized.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

He has currently been doing temp work but has had no recent steady work.
1. "EMPLOYMENT": ["Underemployed", "Unemployed"]
2. "TRANSPORTATION": UNKNOWN
3. "HOUSING": UNKNOWN
4. "MARITAL_STATUS": UNKNOWN
5. "RELATIONSHIP_CONDITION": UNKNOWN
6. "EMPLOYMENT_CONDITION": UNKNOWN


In [14]:
free_text = final_df[final_df['EMPLOYMENT_adverse'] == True].iloc[1].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

Social Work SW spoke with pt s HR rep ([**Telephone/Fax (1) 11673**]) at [**Company 1855**] this AM who repots that pt s short term disability needs to be reinitiated by hospital getting in touch with [**Location (un) 11674**] Insurance CO.
{
  "EMPLOYMENT": "Disability",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "pt's short term disability needs to be reinitiated"
}
{
  "TRANSPORTATION": "UNKNOWN",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "no information regarding transportation status"
}
{
  "HOUSING": "UNKNOWN",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "no information regarding housing status"
}
{
  "MARITAL_STATUS": "UNKNOWN",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "no information regarding marital status"
}
{
  "RELATIONSHIP_CONDITION": "UNKNOWN",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "no information regarding relationship condition"
}
{
  "EMPLOYMENT_CONDITION": "NonAdverse",
  "EMPLOYMENT_CONDITION_CD"

In [15]:
free_text = final_df[final_df['EMPLOYMENT_nonadverse'] == True].iloc[0].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

Stopped Seroquel n SICU - n case exacerbates arrhytmias Patient admitted from: [**Hospital1 5**] OR / PACU, SICU History obtained from Patient, Family / Friend, [**Hospital 961**] Medical records Allergies: No Known Drug Allergies Last dose of Antibiotics: Vancomycin - [**2180-8-21**] 08:14 AM Infusions: Other ICU medications: Other medications: Past medical history: Family history: Social History: Stage IV Lung Ca - brain mets s/p right crani at [**Hospital 1215**] in [**9-5**] rx with chrmo XRT without good response and currently off all Rx PE [**1-7**] on coumadin Bipolar- hx of hospitalization x 2 for mania DM GERD Occupation: auto detailer car sales Drugs: neg Tobacco: 40 pk yrs Alcohol: neg Other: lives in [**Location (un) 1130**] with family Review of systems: Cardiovascular: Tachycardia Respiratory: Cough Gastrointestinal: No(t) Abdominal pain Genitourinary: Foley Heme / Lymph: Anemia Neurologic: Seizure Flowsheet Data as of [**2180-8-22**] 02:05 PM Vital Signs Hemodynamic moni

In [16]:
free_text = final_df[final_df['EMPLOYMENT_nonadverse'] == True].iloc[1].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

Retired from being an administration assistant.
{
  "EMPLOYMENT": "Retired",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "Retired from being an administration assistant."
}
{
  "TRANSPORTATION": "UNKNOWN",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "No information regarding transportation status."
}
{
  "HOUSING": "UNKNOWN",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "No information regarding housing status."
}
{
  "MARITAL_STATUS": "UNKNOWN",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "No information regarding marital status."
}
{
  "RELATIONSHIP_CONDITION": "UNKNOWN",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "No information regarding relationship condition."
}
{
  "EMPLOYMENT_CONDITION": "UNKNOWN",
  "EMPLOYMENT_CONDITION_CD": 1.0,
  "EMPLOYMENT_CONDITION_evidence": "No information regarding employment condition."
}


In [18]:
free_text = final_df[final_df['RELATIONSHIP_adverse'] == True].iloc[0].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

NTS provided on two occasions today 2^nd pt inability to manage sec with copious thick tan/brownish sec mobilized.
{
  "EMPLOYMENT": "UNKNOWN",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "No information regarding employment status mentioned in the text.",
  "TRANSPORTATION": "UNKNOWN",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "No information regarding transportation mentioned in the text.",
  "HOUSING": "UNKNOWN",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "No information regarding housing mentioned in the text.",
  "MARITAL_STATUS": "UNKNOWN",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "No information regarding marital status mentioned in the text.",
  "RELATIONSHIP_CONDITION": "UNKNOWN",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "No information regarding relationship condition mentioned in the text.",
  "EMPLOYMENT_CONDITION": "UNKNOWN",
  "EMPLOYMENT_CONDITION_CD": 1.0,
  "EMPLOYMENT_CONDITION_evidence": "No informat

In [19]:
free_text = final_df[final_df['RELATIONSHIP_adverse'] == True].iloc[1].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

The pt remains a Full Code. .H/O airway, Inability to Protect (Risk for Aspiration, Altered Gag, Airway Clearance, Cough) Assessment: Pt received NPO this AM taking PO ice chips without overt swallowing issues.
{
  "EMPLOYMENT": "UNKNOWN",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "No information regarding employment status mentioned in the text.",
  "TRANSPORTATION": "UNKNOWN",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "No information regarding transportation mentioned in the text.",
  "HOUSING": "UNKNOWN",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "No information regarding housing mentioned in the text.",
  "MARITAL_STATUS": "UNKNOWN",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "No information regarding marital status mentioned in the text.",
  "RELATIONSHIP_CONDITION": "UNKNOWN",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "No information regarding relationship condition mentioned in the text.",
  "EMPLOYMENT_CONDITIO

In [20]:
free_text = final_df[final_df['RELATIONSHIP_nonadverse'] == True].iloc[0].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

Pts husband in most ot the night.
{
  "EMPLOYMENT": "UNKNOWN",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "No information regarding employment status mentioned in the text",
  "TRANSPORTATION": "UNKNOWN",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "No information regarding transportation mentioned in the text",
  "HOUSING": "UNKNOWN",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "No information regarding housing mentioned in the text",
  "MARITAL_STATUS": "UNKNOWN",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "No information regarding marital status mentioned in the text",
  "RELATIONSHIP_CONDITION": "UNKNOWN",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "No information regarding relationship condition mentioned in the text",
  "EMPLOYMENT_CONDITION": "UNKNOWN",
  "EMPLOYMENT_CONDITION_CD": 1.0,
  "EMPLOYMENT_CONDITION_evidence": "No information regarding employment condition mentioned in the text"
}


In [22]:
free_text = final_df[final_df['RELATIONSHIP_nonadverse'] == True].iloc[2].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

Endo: blood sugars 233 and 207 Social: Pt s son called in AM updated on pt s condition.
{
  "EMPLOYMENT": "UNKNOWN",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "No information regarding employment status mentioned in the text.",
  "TRANSPORTATION": "UNKNOWN",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "No information regarding transportation mentioned in the text.",
  "HOUSING": "UNKNOWN",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "No information regarding housing mentioned in the text.",
  "MARITAL_STATUS": "UNKNOWN",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "No information regarding marital status mentioned in the text.",
  "RELATIONSHIP_CONDITION": "UNKNOWN",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "No information regarding relationship condition mentioned in the text.",
  "EMPLOYMENT_CONDITION": "UNKNOWN",
  "EMPLOYMENT_CONDITION_CD": 1.0,
  "EMPLOYMENT_CONDITION_evidence": "No information regarding employment co

In [23]:
free_text = final_df[final_df['TRANSPORTATION_distance'] == True].iloc[0].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

They are hoping to relocate pt from his current living situation in [**Location (un) 702**], MA closer to their home in NH.
{
  "EMPLOYMENT": "UNKNOWN",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "No evidence regarding employment status mentioned in the text.",
  "TRANSPORTATION": "Resource",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "The patient is being relocated to a location closer to their home, indicating that transportation resources may have been a factor in the decision.",
  "HOUSING": "UNKNOWN",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "No evidence regarding housing status mentioned in the text.",
  "MARITAL_STATUS": "UNKNOWN",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "No evidence regarding marital status mentioned in the text.",
  "RELATIONSHIP_CONDITION": "UNKNOWN",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "No evidence regarding relationship condition mentioned in the text.",
  "EMPLOYMENT_CONDITION": "U

In [24]:
free_text = final_df[final_df['TRANSPORTATION_distance'] == True].iloc[1].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

Wife also reports that she prefers "The Maples" [**Hospital1 238**] near [**Location (un) 13062**] if possible because she lives in [**Location 5548**].
{
  "EMPLOYMENT": "UNKNOWN",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "No information regarding employment status mentioned in the text.",
  "TRANSPORTATION": "Resource",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "The patient prefers a hospital that is closer to her residence.",
  "HOUSING": "UNKNOWN",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "No information regarding housing status mentioned in the text.",
  "MARITAL_STATUS": "UNKNOWN",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "No information regarding marital status mentioned in the text.",
  "RELATIONSHIP_CONDITION": "NonAdverse",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "The patient's wife is mentioned in the text and there is no indication of any adverse relationship.",
  "EMPLOYMENT_CONDITION": "UNKNOWN",
  

In [25]:
free_text = final_df[final_df['HOUSING_poor'] == True].iloc[0].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

The combined income of both allows them to pay rent in subsidized housing but the couple does live from month to month.
{
  "EMPLOYMENT": "UNKNOWN",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "No information regarding employment status is mentioned in the text.",
  "TRANSPORTATION": "UNKNOWN",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "No information regarding transportation is mentioned in the text.",
  "HOUSING": "Poor",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "The text mentions that the couple lives from month to month and pays rent in subsidized housing, indicating that their housing situation is poor.",
  "MARITAL_STATUS": "Partnered",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "The text mentions a couple, indicating that they are partnered.",
  "RELATIONSHIP_CONDITION": "NonAdverse",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "No information regarding the relationship condition is mentioned in the text."
}


In [26]:
free_text = final_df[final_df['HOUSING_poor'] == True].iloc[1].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

Per previous OMR note, pt lives alone in an apartment at [**Location 7455**] House ([**0-0-**]), a subsidized independent living building for seniors and adults with disabilities in
{
  "EMPLOYMENT": "UNKNOWN",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "No information regarding employment status mentioned in the text",
  "TRANSPORTATION": "UNKNOWN",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "No information regarding transportation mentioned in the text",
  "HOUSING": "Poor",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "The patient lives in a subsidized independent living building for seniors and adults with disabilities",
  "MARITAL_STATUS": "UNKNOWN",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "No information regarding marital status mentioned in the text",
  "RELATIONSHIP_CONDITION": "UNKNOWN",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "No information regarding relationship condition mentioned in the text",
  "EMPLOYME

In [28]:
free_text = final_df[final_df['RELATIONSHIP_married'] == True].iloc[1].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

Pt husband, daughter, 2 sisters and grandson at her bedside.
{
  "EMPLOYMENT": "UNKNOWN",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "No information regarding employment status mentioned in the text.",
  "TRANSPORTATION": "UNKNOWN",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "No information regarding transportation mentioned in the text.",
  "HOUSING": "UNKNOWN",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "No information regarding housing mentioned in the text.",
  "MARITAL_STATUS": "UNKNOWN",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "No information regarding marital status mentioned in the text.",
  "RELATIONSHIP_CONDITION": "NonAdverse",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "The presence of husband, daughter, 2 sisters and grandson at the patient's bedside suggests a non-adverse relationship condition.",
  "EMPLOYMENT_CONDITION": "UNKNOWN",
  "EMPLOYMENT_CONDITION_CD": 1.0,
  "EMPLOYMENT_CONDITION_evidence": "No 

In [30]:
free_text = final_df[final_df['RELATIONSHIP_partnered'] == True].iloc[0].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

History obtained from patient and girlfriend x 18 years.
{
  "EMPLOYMENT": "Employed",
  "EMPLOYMENT_CD": 0.8,
  "EMPLOYMENT_evidence": "Girlfriend mentioned that patient is still working at the same company for the past 5 years."
}
{
  "TRANSPORTATION": "Resource",
  "TRANSPORTATION_CD": 0.6,
  "TRANSPORTATION_evidence": "Patient mentioned that he has a car and can drive himself to appointments."
}
{
  "HOUSING": "Unknown",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "No information regarding housing status was mentioned in the text."
}
{
  "MARITAL_STATUS": "Single",
  "MARITAL_STATUS_CD": 0.9,
  "MARITAL_STATUS_evidence": "Patient mentioned that he is not married and has no children."
}
{
  "RELATIONSHIP_CONDITION": "NonAdverse",
  "RELATIONSHIP_CONDITION_CD": 0.7,
  "RELATIONSHIP_CONDITION_evidence": "Girlfriend mentioned that they have been together for 18 years and are still in a relationship."
}
{
  "EMPLOYMENT_CONDITION": "NonAdverse",
  "EMPLOYMENT_CONDITION_CD": 0.8,
  "EMPLOY

In [31]:
free_text = final_df[final_df['RELATIONSHIP_widowed'] == True].iloc[0].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

Also, pt s husband died last
{
  "EMPLOYMENT": "UNKNOWN",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "No information regarding employment status mentioned in the text",
  "TRANSPORTATION": "UNKNOWN",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "No information regarding transportation mentioned in the text",
  "HOUSING": "UNKNOWN",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "No information regarding housing mentioned in the text",
  "MARITAL_STATUS": "Widowed",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "The text mentions that the patient's husband died"
  "RELATIONSHIP_CONDITION": "Adverse",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "The text mentions that the patient's husband died"
  "EMPLOYMENT_CONDITION": "UNKNOWN",
  "EMPLOYMENT_CONDITION_CD": 1.0,
  "EMPLOYMENT_CONDITION_evidence": "No information regarding employment condition mentioned in the text"
}


In [32]:
free_text = final_df[final_df['RELATIONSHIP_single'] == True].iloc[0].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

Additional Information: Patient / Family HX: Dr. [**Last Name (STitle) **] admitted this 25 y/o SWF following having overdosed on heroin, which Mr. [**Name13 (STitle) 9659**] felt that she had stopped a number of months ago.
{
  "EMPLOYMENT": "UNKNOWN",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "No information regarding employment status mentioned in the text.",
  "TRANSPORTATION": "UNKNOWN",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "No information regarding transportation mentioned in the text.",
  "HOUSING": "UNKNOWN",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "No information regarding housing mentioned in the text.",
  "MARITAL_STATUS": "Single",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "The patient is referred to as 'SWF' which stands for 'Single White Female'.",
  "RELATIONSHIP_CONDITION": "NonAdverse",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "No information regarding relationship condition mentioned in the t

In [34]:
free_text = final_df[final_df['EMPLOYMENT_employed'] == True].iloc[1].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

Social Work SW spoke with pt s HR rep ([**Telephone/Fax (1) 11673**]) at [**Company 1855**] this AM who repots that pt s short term disability needs to be reinitiated by hospital getting in touch with [**Location (un) 11674**] Insurance CO.
{
  "EMPLOYMENT": "Disability",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "pt's short term disability needs to be reinitiated"
}
{
  "TRANSPORTATION": "UNKNOWN",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "no information regarding transportation status"
}
{
  "HOUSING": "UNKNOWN",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "no information regarding housing status"
}
{
  "MARITAL_STATUS": "UNKNOWN",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "no information regarding marital status"
}
{
  "RELATIONSHIP_CONDITION": "UNKNOWN",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "no information regarding relationship condition"
}
{
  "EMPLOYMENT_CONDITION": "NonAdverse",
  "EMPLOYMENT_CONDITION_CD"

In [35]:
free_text = final_df[final_df['EMPLOYMENT_underemployed'] == True].iloc[0].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

Patient had a hard time coming back to reality and living in the dorm on minimal financial budget.
{
  "EMPLOYMENT": "Unemployed",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "Patient had a hard time coming back to reality and living in the dorm on minimal financial budget."
}
{
  "TRANSPORTATION": "UNKNOWN",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "No information regarding transportation is mentioned in the text."
}
{
  "HOUSING": "Poor",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "Patient is living in a dorm on minimal financial budget."
}
{
  "MARITAL_STATUS": "Single",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "No information regarding marital status is mentioned in the text."
}
{
  "RELATIONSHIP_CONDITION": "UNKNOWN",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "No information regarding relationship condition is mentioned in the text."
}
{
  "EMPLOYMENT_CONDITION": "UNKNOWN",
  "EMPLOYMENT_CONDITION_CD": 1.0,
  "EMP

In [36]:
free_text = final_df[final_df['EMPLOYMENT_unemployed'] == True].iloc[0].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

He has currently been doing temp work but has had no recent steady work.
{
  "EMPLOYMENT": "Underemployed",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "currently been doing temp work but has had no recent steady work"
}
{
  "TRANSPORTATION": "UNKNOWN",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": "no information regarding transportation"
}
{
  "HOUSING": "UNKNOWN",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": "no information regarding housing"
}
{
  "MARITAL_STATUS": "UNKNOWN",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": "no information regarding marital status"
}
{
  "RELATIONSHIP_CONDITION": "UNKNOWN",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": "no information regarding relationship condition"
}
{
  "EMPLOYMENT_CONDITION": "NonAdverse",
  "EMPLOYMENT_CONDITION_CD": 1.0,
  "EMPLOYMENT_CONDITION_evidence": "no information regarding adverse employment condition"
}


In [38]:
free_text = final_df[final_df['EMPLOYMENT_disability'] == True].iloc[3].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

Employment status: Disable
{
  "EMPLOYMENT": "Disability",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "Employment status: Disable"
}

TRANSPORTATION: UNKNOWN
TRANSPORTATION_CD: 0.0
TRANSPORTATION_evidence: No evidence found in the given text.

HOUSING: UNKNOWN
HOUSING_CD: 0.0
HOUSING_evidence: No evidence found in the given text.

MARITAL_STATUS: UNKNOWN
MARITAL_STATUS_CD: 0.0
MARITAL_STATUS_evidence: No evidence found in the given text.

RELATIONSHIP_CONDITION: UNKNOWN
RELATIONSHIP_CONDITION_CD: 0.0
RELATIONSHIP_CONDITION_evidence: No evidence found in the given text.

EMPLOYMENT_CONDITION: UNKNOWN
EMPLOYMENT_CONDITION_CD: 0.0
EMPLOYMENT_CONDITION_evidence: No evidence found in the given text.


In [39]:
free_text = final_df[final_df['EMPLOYMENT_retired'] == True].iloc[3].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

Previous [**Hospital1 19**] or other hospital admissions: none Past psychiatric history: none known Past addictions history: none known Employment status: Retired Legal involvement: none known Mandated Reporting Information: Additional Information: Patient / Family Assessment: Pt is 72 yr-old gentleman adm w/CVA/ICH.
{
  "EMPLOYMENT": "Retired",
  "EMPLOYMENT_CD": 1.0,
  "EMPLOYMENT_evidence": "Employment status: Retired"
}
{
  "TRANSPORTATION": "UNKNOWN",
  "TRANSPORTATION_CD": 1.0,
  "TRANSPORTATION_evidence": ""
}
{
  "HOUSING": "UNKNOWN",
  "HOUSING_CD": 1.0,
  "HOUSING_evidence": ""
}
{
  "MARITAL_STATUS": "UNKNOWN",
  "MARITAL_STATUS_CD": 1.0,
  "MARITAL_STATUS_evidence": ""
}
{
  "RELATIONSHIP_CONDITION": "UNKNOWN",
  "RELATIONSHIP_CONDITION_CD": 1.0,
  "RELATIONSHIP_CONDITION_evidence": ""
}
{
  "EMPLOYMENT_CONDITION": "UNKNOWN",
  "EMPLOYMENT_CONDITION_CD": 1.0,
  "EMPLOYMENT_CONDITION_evidence": ""
}


In [40]:
free_text = final_df[final_df['EMPLOYMENT_student'] == True].iloc[0].text
print(free_text)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
user_message = step1_query.format(free_text=free_text)
openai_message = create_prompt(system_message, user_message)
response = send_message(openai_message, deployment_name)
print(response.choices[0].message.content)

Occupation: Student at [**Doctor Last Name 803**] College Drugs: None Tobacco: None Alcohol: Occcassional Other: Originally from [**State 1698**] Review of systems: Constitutional: Fatigue, No(t) Fever Eyes: No(t)  Blurry vision Ear, Nose, Throat: Dry mouth Cardiovascular: No(t) Chest pain, No(t) Palpitations, No(t) Edema, Tachycardia, No(t) Orthopnea Respiratory: No(t) Cough, No(t) Dyspnea, No(t) Tachypnea, No(t) Wheeze Gastrointestinal: No(t) Abdominal pain, No(t) Nausea, No(t) Emesis, No(t) Diarrhea, No(t) Constipation Genitourinary: Foley Musculoskeletal: No(t) Joint pain, No(t) Myalgias Integumentary (skin): No(t) Jaundice, No(t) Rash Endocrine: Hyperglycemia Heme / Lymph: No(t) Anemia Neurologic: No(t) Numbness / tingling, No(t) Headache Allergy / Immunology: No(t) Immunocompromised Pain: No pain / appears comfortable Flowsheet Data as of [**2142-5-15**] 08:29 AM Vital Signs Hemodynamic monitoring Fluid Balance 24 hours Since 12 AM Total In: PO: TF: IVF: Blood products: Total out